<a href="https://colab.research.google.com/github/NUELBUNDI/PDS_PROJECT/blob/main/TimeSeriesPaper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Time Series Project Paper**

---
## Student Name :Lee Bundi
## Student No:  :102586

The purpose of this paper is to model a time series analysis. 
Time series is a set of observations, each one being recorded at a specific time.
Time series analysis is done on a time series data.
In this paper our focus will be on univarite time series (Meaning i will use a single set of observation or variable indexed over time).

The **basic objective**  is to determine a model that describes the pattern of the time series. Uses for such a model are:

1. To describe the important features of the time series pattern.
2. To explain how the past affects the future.
3. To forecast future values of the series.


The time series data i'm using in this paper comprise of annual average temperature in kenya for the period starting 1981 to 2020. The source of the data is https://africaopendata.org/dataset


##Steps to Time Series Analysis.


*   Visualize the time series plot.
*   Check for stationarity.
*   Make the time series stationary.
*   Plot the ACF and PCF.
*   Select the model and train the data.
*   Choose the best performing model.
*   Forecast using the model choosen on the test data
*   Analysis the Forecast performance- iterate through the step until you find the best forecast performance.
*   Perform Future forecast.











In [ ]:
# Import Packages

%matplotlib inline
from bokeh.io import output_notebook
from bokeh import models, palettes, transform
from bokeh.plotting import figure, show
import pandas as pd
import pandas_profiling
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn import cluster, decomposition, pipeline, preprocessing
import statsmodels
import missingno as mn
import plotly.offline as py
import plotly.express as px
import datetime
import plotly.graph_objects as go
from statsmodels.tsa.stattools import adfuller,acf
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf ,plot_pacf
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm
from pylab import rcParams
from math import sqrt
from sklearn.metrics import mean_squared_error
from pmdarima import auto_arima
import warnings
warnings.filterwarnings('ignore')
import statsmodels.api as sm
from statsmodels.tsa.ar_model import AutoReg, ar_select_order


Read the data set and load it to pandas data frame

In [ ]:

# df=pd.read_csv('https://raw.githubusercontent.com/NUELBUNDI/PDS_PROJECT/main/SalesPerMonthWst.csv')
# df=pd.read_csv("https://raw.githubusercontent.com/NUELBUNDI/PDS_PROJECT/main/climate.csv")
df=pd.read_csv("https://raw.githubusercontent.com/NUELBUNDI/PDS_PROJECT/main/tempkenyadata.csv",index_col=0,parse_dates=True)
# df.columns=['date','temp']
x=df.values
# print(x)
df.head()

##**Plot the time series data to visualize and analysis.**





In [ ]:
df.plot(figsize=(12,6))
plt.xlabel('Year')
plt.ylabel('Annually Avearage Temperature ')
plt.title('Trend of the Time Series')
plt.show()

Decompose a time series.

This enable us to visualize the components of time series namely:


*   Trend-       Increasing or decreasing value in the series.

*   seasonarity- Any repeating cycle
*   Noise        Random Variation in the series

*   Level       Average value in the series



In [ ]:
plt.figure(figsize=(15,5))
result = seasonal_decompose(df, model='additive')
result.plot()
plt.show()


# The series as no seasonality

### **Checking For Stationarity**

In [ ]:
# Function to check for stationarity

def stationarity_test(timeseries):

    print('RESULTS OF DICKEY-FULLER TEST\n')
    df_test = adfuller(timeseries,autolag='AIC')
    df_output = pd.Series(df_test[0:4], index = ['Test Statistic', 'p-value', '#Lags Used', 'Number of Observations Used'])
    for key, value in df_test[4].items():
        df_output['Critical Value (%s)' %key] = value
    print(df_output)
    print("****************************************************")
    print(f'INFERENCE:         THE TIME SERIES IS {"NON-" if df_test[1]>=0.05 else ""}STATIONARY')

stationarity_test(df)

### **Make Non-stationary time series data stationary by differencing**

Since from the previous results the time series is non-stationary we have to make stationary first, this is because we can not perform ARIMA models to non-stationary time series data.



In [ ]:
# Check the First Order Difference
df_diff=df.diff().dropna()

stationarity_test(train_data_diff)
# df_diff.plot()

##**Plot ACF and PACF**

Plot ACF and PACF of the stationary data this is useful in helping start to choose our model parameters (p,d,q) in our ARIMA models;


*   P-The number of lag observations included in the AR model.
*   D-Degree of differencing
*   Q-Moving average of MA

In [ ]:
#Determine P and Q by Plotting the ACF AND PACF

def plot_acf_pcf(ts_data):
  plot_pacf(df['readig'],lags=10)
  plot_acf(df['readig'],lags=10)

plot_acf_pcf(df_diff)



Split time series data set into two : the train and test data set.

In [ ]:
# Split the data into train and test
train=df.iloc[:31]
test=df.iloc[31:]

##**AR Model**




Fit AR model of lag 3 since PACF significant spike are 3 in Number.

In [ ]:
train_diff=train.diff().dropna()
test_diff=test.diff().dropna()
ar_model = AutoReg(train_diff, lags=3).fit()
print(ar_model.summary())

**Intepretation**


1.   The AIC is 1.447 its ideal since its small
2.   Apart from lag 3 all other lags are significant since the p values is less that 0.05


Using the above model lets forecast on the test data and calculate the error


In [ ]:
start=len(train_diff)
end=len(train_diff)+len(test)-1
predict = ar_model.predict(start=start, end=end, dynamic=False)
# print(predict)

# Plot the prediction vs test data

# pyplot.plot(pred)
# pyplot.plot(test, color='red')

predict.plot(legend=True)
test_diff['readig'].plot(legend=True)
# rmse=sqrt(mean_squared_error(predict,test_diff['readig']))
# plt.title(f'The RMSE IS {rmse}')
# print(test['readig'].mean())


In [ ]:
# Calculate Performance 

from math import sqrt
from sklearn.metrics import mean_squared_error
rmse=sqrt(mean_squared_error(test_diff,predict))
print(rmse)

In [ ]:
# Make future predictions

pred_future = ar_model.predict(start=start+1, end=end+5, dynamic=False)
print('The future predictions of the next 5 years')
print(pred_future)


In [ ]:
# series = df["Number of Bookings"]
# new_series = np.log(series).diff()
# # getting only the value of zeroth index since the diff() operation looses first value.
# new_series.iloc[0] = np.log(series.iloc[0])
# result = np.exp(new_series.cumsum())

In [ ]:
from statsmodels.tsa.ar_model import AutoReg, ar_select_order
ar_model = AutoReg(train_data_diff, lags=3).fit()
print(ar_model.summary())

### **Build an ARIMA model .**

In [ ]:
# #How to do find the optimal ARIMA model manually using Out-of-Time Cross validation

# # Build Model
# # model = ARIMA(train, order=(3,2,1))  
# # order=(2,1,2)
# # model=ARIMA(endog=train_data,order=order)

# model=ARIMA(endog=train_data, order=(1, 1, 0))  
# fitted = model.fit(disp=-1)  

# # Forecast
# fc, se, conf = fitted.forecast(22, alpha=0.05)  # 95% conf

# # Make as pandas series
# fc_series = pd.Series(fc, index=test_data.index)
# lower_series = pd.Series(conf[:, 0], index=test_data.index)
# upper_series = pd.Series(conf[:, 1], index=test_data.index)

# # Plot
# plt.figure(figsize=(12,5), dpi=100)
# plt.plot(train_data, label='training')
# plt.plot(test_data, label='actual')
# plt.plot(fc_series, label='forecast')
# plt.fill_between(lower_series.index, lower_series, upper_series, 
#                  color='k', alpha=.15)
# plt.title('Forecast vs Actuals')
# plt.legend(loc='upper left', fontsize=8)
# plt.show()


In [ ]:
# # Accuracy metrics
# def forecast_accuracy(forecast, actual):
#     mape = np.mean(np.abs(forecast - actual)/np.abs(actual))  # MAPE
#     me = np.mean(forecast - actual)             # ME
#     mae = np.mean(np.abs(forecast - actual))    # MAE
#     mpe = np.mean((forecast - actual)/actual)   # MPE
#     rmse = np.mean((forecast - actual)**2)**.5  # RMSE
#     corr = np.corrcoef(forecast, actual)[0,1]   # corr
#     mins = np.amin(np.hstack([forecast[:,None], 
#                               actual[:,None]]), axis=1)
#     maxs = np.amax(np.hstack([forecast[:,None], 
#                               actual[:,None]]), axis=1)
#     minmax = 1 - np.mean(mins/maxs)             # minmax
#     acf1 = acf(fc-test_data)[1]                      # ACF1
#     return({'MAPE':mape, 'ME':me, 'MAE': mae, 
#             'MPE': mpe, 'RMSE':rmse, 'ACF1':acf1, 
#             'CORR':corr, 'MINMAX':minmax})

# summary=forecast_accuracy(fc, test_data.values)

# summary


In [ ]:

stepwise_fit=auto_arima(df['readig'],trace=True,supprese_warnings=True)

In [ ]:
# Train model
import statsmodels.api as sm

arima_model = sm.tsa.arima.ARIMA(train['readig'], order=(2,1,0))
arima_model = arima_model.fit()
arima_model.summary()

# model=ARIMA(train,order=(0,1,0))
# model=model.fit()
# model.summary()

In [ ]:
# Make Predict on Test set

start=len(train)
end=len(train)+len(test)-1
pred=arima_model.predict(start=start,end=end,typ='levels')
print(pred)
# pred.index=df.index[start:end+1]

In [ ]:
# Plot predict vs Actual

pred.plot(legend=True)
test['readig'].plot(legend=True)
rmse=sqrt(mean_squared_error(pred,test['readig']))
plt.title(f'The RMSE IS {rmse}')
print(test['readig'].mean())

In [ ]:
# If the model is okay then train the model on the whole data set and make future predictions

arima_model = sm.tsa.arima.ARIMA(df['readig'], order=(2,1,0))
arima_model = arima_model.fit()
df.tail()



In [ ]:
# For Future Dates

# index_future_dates=pd.date_range(start='2020-12-31', end ='2030-12-31')
index_future_dates=pd.date_range(start=pd.datetime(2020, 12, 31), periods=11, freq=pd.DateOffset(years=1))
# print(index_future_dates)
pred2=arima_model.predict(start=len(df),end=len(df)+10,typ='levels').rename('ARIMA PREDICTIONS')
# print(pred2)
# pred2.index=index_future_dates
print(pred2)


In [ ]:
pred2.plot(figsize=(10,6),legend=True)

###Conclusion

In [ ]:
# https://www.machinelearningplus.com/resources/arima/arima-forecast-test-results/
# https://www.machinelearningplus.com/resources/arima/implement-arima-model/
# https://www.machinelearningplus.com/time-series/time-series-analysis-python/
# https://www.machinelearningplus.com/time-series/arima-model-time-series-forecasting-python/
# pip install statsmodels==0.11.0
# https://www.kaggle.com/satishgunjal/tutorial-time-series-analysis-and-forecasting
# https://www.geeksforgeeks.org/python-arima-model-for-time-series-forecasting/